In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import requests
from urllib.request import urlopen
from tqdm import tqdm
import time
import csv
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
def getTables(response):
    # HTTPステータスコードが200（成功）の場合のみ処理を続行
    if response.getcode() == 200:
        # HTMLをパースしてBeautifulSoupオブジェクトを作成
        bs = BeautifulSoup(response, 'html.parser')
        bs = bs.decode('UTF-8')
        html_string = str(bs)
        html_io = StringIO(html_string)

        # テーブルデータを抽出
        tables = pd.read_html(html_io)
        df = tables[0]
        print(df)
        return df

    else:
        print(f"HTTPステータスコード {response.getcode()}: ページの取得に失敗しました")
        

In [ ]:
#開催年
years = [str(i).zfill(4) for i in range(2021, 2022)]
#開催場所 01:札幌, 02:函館, 03:福島, 04:新潟, 05:東京, 06:中山, 07:中京, 08:京都, 09::阪神, 10:小倉
places = [str(i).zfill(2) for i in range (5, 6)]
#開催回
times = [str(i).zfill(2) for i in range(1, 2)]
#開催日
days = [str(i).zfill(2) for i in range(1, 2)]
#レースNo
races = [str(i).zfill(2) for i in range(1, 13)]

raceIdList = []
for y in years:
    for p in places:
        for t in times:
            for d in days:
                for r in races:
                    raceIdList.append(y + p + t + d + r)
                    

In [ ]:
url = 'https://db.netkeiba.com/race/'
colName = ['raceId', 'htmlBytes']
df = pd.DataFrame(columns=colName)
escapeList = []
if (os.path.isfile('race_html.pkl')):
    df = pd.read_pickle('race_html.pkl')
    escapeList = df['raceId'].to_list()

def addEscapeList(id :str, ll :list):
    idAry = [id[0:4], id[4:6], id[6:8], id[8:10], id[10:12]]
    for r in range(1,13):
        idAry[4] = str(r).zfill(2)
        ll.append(''.join(idAry))
    if idAry[3]=='01':
        for d in range(2,9):
            idAry[3] = str(d).zfill(2)
            ll = addEscapeList(''.join(idAry),ll)
    if idAry[2]=='01':
        for t in range(2,9):
            idAry[2] = str(t).zfill(2)
            ll= addEscapeList(''.join(idAry), ll)
    
    return ll

for raceId in tqdm(raceIdList):
    try:
        if raceId in escapeList:
            continue
        response = url + raceId
        html = requests.get(response)
        soup = BeautifulSoup(html.content, 'html.parser')

        if 'レース結果' in soup.text:
            tmpDf = pd.DataFrame([[raceId, html.content]], colmuns=colName)
            df = pd.concat([df,tmpDf],axis=0, ignore_index=True)
        else:
            escaprList = addEscapeList(raceId, escapeList)
        time.sleep(3)
    except:
        print('exception catch')
        break

df.to_pickle('race_html')
   # race_data = getTables(response)
    #race_data['raceId'] = [raceId] * len(race_data)
    
     
    #time.sleep(3)

#print(df)

In [ ]:
df.to_csv('data.csv',encoding='utf-8')
df.columns
print(df)